# BÁO CÁO BÀI TẬP LẬP TRÌNH 1: HIỂU DỮ LIỆU  
## Môn học: Khai phá Dữ liệu (Data Mining)

---

## 👥 Thông tin nhóm:
1. **Họ và tên:** Lường Minh Trí  
   **Mã sinh viên:** 23020440  
   **Lớp:** K68A-AI2  

2. **Họ và tên:** Ngô Quang Dũng  
   **Mã sinh viên:** 23020440  
   **Lớp:** K68A-AI2  

3. **Họ và tên:** Kiều Đức Nam  
   **Mã sinh viên:** 23020440  
   **Lớp:** K68A-AI2  

**📊 Data set lựa chọn:**  
`[Simplize_VNM_FinancialIndicator_20250315, Simplize_VNM_PriceHistory_20250315]`
## 🎯 Mục tiêu bài tập

Bài tập nhằm giúp sinh viên **hiểu và khám phá dữ liệu tài chính** thực tế của doanh nghiệp niêm yết trên sàn chứng khoán Việt Nam, thông qua:
- Phân tích thống kê mô tả.  
- Trực quan hóa dữ liệu để phát hiện xu hướng biến động.  
- Tiền xử lý và kết hợp dữ liệu từ nhiều nguồn.  
- Tính toán và phân tích tương quan giữa giá cổ phiếu và các chỉ số tài chính.  

Mục tiêu cuối cùng là hình thành kỹ năng **hiểu dữ liệu (data understanding)** và **phân tích mối quan hệ tài chính – giá cổ phiếu** trong ngữ cảnh doanh nghiệp cụ thể.

---

## 🧩 Các phần thực hiện trong bài

### **1. Quan sát để hiểu doanh nghiệp và dữ liệu**
- Thực hiện thống kê miêu tả hai bảng dữ liệu (Chỉ số tài chính & Lịch sử giá cổ phiếu):  
  - Tính **five-number summary** (min, Q1, median/Q2, Q3, max).  
  - Tính **trung bình, trung vị, độ lệch chuẩn, phương sai**.  
- Vẽ **biểu đồ biến động giá cổ phiếu** theo thời gian (sử dụng giá đóng cửa mỗi ngày).  
- Quan sát và nhận xét các đợt **tăng/giảm bất thường**, tìm hiểu **bối cảnh thị trường hoặc sự kiện công ty** có thể ảnh hưởng.

---

### **2. Tiền xử lý dữ liệu**
- Tạo một bảng dữ liệu mới bằng cách kết hợp **bảng Lịch sử giá** và **bảng Chỉ số tài chính**.  
- Các cột bao gồm:  
  - **Thời gian (Quý/Năm)**, **Giá đóng cửa trung bình**, **Thay đổi giá**, **% thay đổi**  
  - Các chỉ số tài chính quan trọng:  
    - Biên lợi nhuận gộp  
    - Biên lợi nhuận ròng  
    - P/E  
    - EPS  
    - Tăng trưởng EPS  
    - ROE  
    - Tỷ lệ Nợ phải trả/Vốn chủ sở hữu  
    - Khả năng thanh toán tổng quát  
    - Vòng quay tài sản  
    - Giá trị sổ sách  
- Mỗi hàng thể hiện dữ liệu của **một quý**, trong đó:
  - Giá cổ phiếu = **trung bình của 29 ngày quanh ngày cuối quý** (±14 ngày).  
  - Các cột thay đổi giá và % thay đổi được tính lại tương ứng.

---

### **3. Lựa chọn các yếu tố nguy cơ (Risk Factors) tiềm năng**
- Phân tích các **yếu tố nội tại** (từ chỉ số tài chính) có thể ảnh hưởng đến giá cổ phiếu.  
- Tính **độ tương quan Pearson** giữa các chỉ số tài chính và giá cổ phiếu.  
- Đưa ra **nhận xét và kết luận**:
  - Chỉ số nào có mối quan hệ mạnh nhất (thuận hoặc nghịch).  
  - Liên hệ với thực tế doanh nghiệp.

---

### **4. Kết quả & Báo cáo**
- Trình bày code, hiển thị 5 dòng đầu của bảng dữ liệu sau khi tiền xử lý.  
- Biểu đồ phải có **tên, trục, chú thích đầy đủ**.  
- Phân tích thống kê, biểu đồ và ma trận tương quan.  
- Đưa ra **nhận xét tổng hợp**:  
  - Dữ liệu phản ánh điều gì về doanh nghiệp.  
  - Các chỉ số tài chính nào ảnh hưởng nhiều nhất tới giá cổ phiếu.

---

### ⚙️ Môi trường thực hiện
- Sử dụng **Docker Container** cho môi trường Big Data & Data Mining.  
- Ngôn ngữ lập trình: **Python 3.x**  
- Thư viện: `pandas`, `numpy`, `matplotlib`, `seaborn`, `scipy`

---

📅 **Thời gian thực hiện:** 2021–2025 (dữ liệu trong 5 năm gần đây)  
📈 **Nguồn dữ liệu:** Simplize.vn (bộ dữ liệu giá cổ phiếu & chỉ số tài chính)

---

> 🧭 *Mục tiêu cuối cùng:*  
> Hiểu rõ cách dữ liệu tài chính phản ánh hoạt động kinh doanh và biến động thị trường, đồng thời rèn luyện kỹ năng phân tích dữ liệu thực tế với Python.
